In [1]:
import pandas as pd
import numpy as np
import scraping
import os
import string
from sklearn import preprocessing

In [2]:
data = []
nonStandardName = []
duplicates = [] 
notFound = [] 
errors = []

for file in os.listdir('data'):
    t1, t2, t3, t4, t5 = np.load('data/' + file, allow_pickle=True)
    data.extend(t1)
    nonStandardName.extend(t2)
    duplicates.extend(t3)
    notFound.extend(t4)
    errors.extend(t5)

## Data Cleaning
*Most players are scraped properly, but there are 4 types of exceptions found.*
1. **nonStandardName** - These are the players with middle names, punctuation within their names, etc.
2. **duplicates** - These are the players with the same names with the same final season. It is hard to differentiate the players using the code but easy by the eye
3. **notFound** - These are players who didn't play in the NCAA and entered the draft
4. **errors** - Any other errors

In [3]:
len(data)

1435

**1. nonStandardName**

In [4]:
nonStandardName

[[2005, 'Alejandro Carmona Sanchez', 22, 'Undrafted', 80],
 [2006, 'Marcus D. Williams', 20, 'Top30', 22],
 [2007, 'Marcus E. Williams', 20, 'Top45', 33],
 [2008, 'Luc Mbah a Moute', 21, 'Top45', 37],
 [2009, 'Nando De Colo', 22, 'Top60', 53],
 [2010, 'Anthony Mason, Jr.', 23, 'Undrafted', 80],
 [2013, 'Tim Hardaway Jr.', 21, 'Top30', 24],
 [2013, 'Trey McKinney Jones', 22, 'Undrafted', 80],
 [2014, 'Walt Lemon, Jr.', 21, 'Undrafted', 80],
 [2015, 'Kelly Oubre, Jr.', 19, 'Top15', 15],
 [2015, 'Larry Nance, Jr.', 22, 'Top30', 27],
 [2015, 'Derek Cooke, Jr.', 23, 'Undrafted', 80],
 [2016, 'Wade Baldwin IV', 20, 'Top30', 17],
 [2016, 'Gary Payton II', 23, 'Undrafted', 80],
 [2016, 'James Webb III', 22, 'Undrafted', 80],
 [2017, 'Derrick Walton, Jr.', 22, 'Undrafted', 80],
 [2017, 'James Blackmon, Jr.', 22, 'Undrafted', 80],
 [2017, 'George de Paula', 21, 'Undrafted', 80],
 [2018, 'Marvin Bagley III', 19, 'Top15', 2],
 [2018, 'Jaren Jackson, Jr.', 18, 'Top15', 4],
 [2018, 'Wendell Carter, 

In [6]:
# Urls obtained manually
urls = ['https://www.sports-reference.com/cbb/players/marcus-williams-1.html', 
        'https://www.sports-reference.com/cbb/players/marcus-williams-2.html',
        'https://www.sports-reference.com/cbb/players/luc-richard-mbah-a-moute-1.html',
        'https://www.sports-reference.com/cbb/players/anthony-mason-2.html',
        'https://www.sports-reference.com/cbb/players/tim-hardaway-jr-1.html',
        'https://www.sports-reference.com/cbb/players/trey-mckinney-jones-1.html', 
        'https://www.sports-reference.com/cbb/players/walt-lemon--1.html',
        'https://www.sports-reference.com/cbb/players/kelly-oubre-1.html',
        'https://www.sports-reference.com/cbb/players/larry-nance-2.html',
        'https://www.sports-reference.com/cbb/players/derek-cooke-1.html',
        'https://www.sports-reference.com/cbb/players/wade-baldwin-1.html',
        'https://www.sports-reference.com/cbb/players/gary-payton-2.html',
        'https://www.sports-reference.com/cbb/players/james-webb-1.html',
        'https://www.sports-reference.com/cbb/players/derrick-walton-1.html',
        'https://www.sports-reference.com/cbb/players/james-blackmon-1.html',
        'https://www.sports-reference.com/cbb/players/marvin-bagleyiii-1.html',
        'https://www.sports-reference.com/cbb/players/jaren-jacksonjr-1.html',
        'https://www.sports-reference.com/cbb/players/wendell-carterjr-1.html',
        'https://www.sports-reference.com/cbb/players/michael-porterjr-1.html',
        'https://www.sports-reference.com/cbb/players/troy-brown-5.html',
        'https://www.sports-reference.com/cbb/players/melvin-frazier-1.html',
        'https://www.sports-reference.com/cbb/players/gary-trentjr-1.html',
        'https://www.sports-reference.com/cbb/players/bonzie-colson-1.html',
        'https://www.sports-reference.com/cbb/players/kevin-porterjr-1.html',
        'https://www.sports-reference.com/cbb/players/barry-brown-1.html',
        'https://www.sports-reference.com/cbb/players/anthony-lawrencejr-1.html',
        'https://www.sports-reference.com/cbb/players/galen-robinsonjr-1.html',
        'https://www.sports-reference.com/cbb/players/james-palmer-1.html']

invalid = ['Alejandro Carmona Sanchez', 'Nando De Colo', 'George de Paula', 'Bruce Brown, Jr.']
newList = []
for row in nonStandardName:
    if row[1] in invalid:
        continue
    else:
        newList.append(row)

out = []
for i in range(len(urls)):
    player = newList[i]
    name, age, labelCat, labelPick = player[1], player[2], player[3], player[4]
    url = urls[i]
    row = scraping.scrapeRow(url)
    out.append([row[0], name, age] + row[2:] + [labelCat, labelPick])

In [7]:
data.extend(out)
len(data)

1463

**2. duplicates**

In [8]:
duplicates

[[2005, 'David Lee', 22, 'Top30', 30],
 [2007, 'Carl Landry', 23, 'Top45', 31],
 [2008, 'Mike Taylor', 22, 'Top60', 55],
 [2008, 'Ivan Johnson', 24, 'Undrafted', 80],
 [2010, 'Jamar Smith', 23, 'Undrafted', 80],
 [2011, 'Tristan Thompson', 20, 'Top15', 4],
 [2011, 'Chris Wright', 22, 'Undrafted', 80],
 [2011, 'Chris Wright', 21, 'Undrafted', 80],
 [2012, 'Dion Waiters', 20, 'Top15', 4],
 [2012, 'Chris Johnson', 22, 'Undrafted', 80],
 [2014, 'Joe Harris', 22, 'Top45', 33],
 [2015, 'Joe Young', 22, 'Top45', 43],
 [2015, 'Marcus Thornton', 22, 'Top45', 45],
 [2015, 'Greg Whittington', 22, 'Undrafted', 80],
 [2015, 'Chris Walker', 20, 'Undrafted', 80],
 [2016, 'Derrick Jones', 19, 'Undrafted', 80],
 [2017, 'Dennis Smith', 19, 'Top15', 9],
 [2017, 'Andrew White', 24, 'Undrafted', 80],
 [2018, 'Zach Smith', 22, 'Undrafted', 80]]

In [9]:
# Urls obtained manually
urls = ['https://www.sports-reference.com/cbb/players/david-lee-1.html',
        'https://www.sports-reference.com/cbb/players/carl-landry-1.html',
        'https://www.sports-reference.com/cbb/players/tristan-thompson-2.html',
        'https://www.sports-reference.com/cbb/players/chris-wright-1.html',
        'https://www.sports-reference.com/cbb/players/chris-wright-2.html',
        'https://www.sports-reference.com/cbb/players/dion-waiters-1.html',
        'https://www.sports-reference.com/cbb/players/chris-johnson-1.html',
        'https://www.sports-reference.com/cbb/players/joe-harris--1.html',
        'https://www.sports-reference.com/cbb/players/joseph-young-1.html',
        'https://www.sports-reference.com/cbb/players/marcus-thornton-3.html',
        'https://www.sports-reference.com/cbb/players/chris-walker-6.html',
        'https://www.sports-reference.com/cbb/players/derrick-jonesjr-1.html',
        'https://www.sports-reference.com/cbb/players/dennis-smithjr-1.html',
        'https://www.sports-reference.com/cbb/players/andrew-whiteiii-1.html',
        'https://www.sports-reference.com/cbb/players/zach-smith-1.html']

invalid = ['Mike Taylor', 'Ivan Johnson', 'Jamar Smith', 'Greg Whittington']
newList = []
for row in nonStandardName:
    if row[1] in invalid:
        continue
    else:
        newList.append(row)

out = []
for i in range(len(urls)):
    player = newList[i]
    name, age, labelCat, labelPick = player[1], player[2], player[3], player[4]
    url = urls[i]
    row = scraping.scrapeRow(url)
    out.append([row[0], name, age] + row[2:] + [labelCat, labelPick])

In [10]:
data.extend(out)
len(data)

1478

**3. notFound**

In [11]:
notFound

[[2005, 'Martell Webster', 18, 'Top15', 6],
 [2005, 'Andrew Bynum', 17, 'Top15', 10],
 [2005, 'Fran Vazquez', 22, 'Top15', 11],
 [2005, 'Yaroslav Korolev', 18, 'Top15', 12],
 [2005, 'Gerald Green', 19, 'Top30', 18],
 [2005, 'Johan Petro', 19, 'Top30', 25],
 [2005, 'Ian Mahinmi', 18, 'Top30', 28],
 [2005, 'C.J. Miles', 18, 'Top45', 34],
 [2005, 'Ricky Sanchez', 17, 'Top45', 35],
 [2005, 'Ersan Ilyasova', 18, 'Top45', 36],
 [2005, 'Monta Ellis', 19, 'Top45', 40],
 [2005, 'Roko Ukic', 20, 'Top45', 41],
 [2005, 'Mile Ilic', 21, 'Top45', 43],
 [2005, 'Martynas Andriuskevicius', 19, 'Top45', 44],
 [2005, 'Mickael Gelabale', 22, 'Top60', 48],
 [2005, 'Andray Blatche', 18, 'Top60', 49],
 [2005, 'Axel Hervelle', 22, 'Top60', 52],
 [2005, 'Marcin Gortat', 21, 'Top60', 57],
 [2005, 'Uros Slokar', 22, 'Top60', 58],
 [2005, 'Cenk Akyol', 18, 'Top60', 59],
 [2005, 'Marcelinho Huertas', 22, 'Undrafted', 80],
 [2005, 'Deji Akindele', 22, 'Undrafted', 80],
 [2005, "D'or Fischer", 23, 'Undrafted', 80],


In [ ]:
# Most of the above have not played in NCAA, but some are errors
# This function returns all the players on sports-reference that have final seasons between 2005 and 2019, given
# their last name
def getPlayersHTML(lastName):
    url = f"https://www.sports-reference.com/cbb/players/{lastName}-index.html"
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    allrows = soup.find_all('p')
    out = []
    for row in allrows:
        if row.find('small') is not None:
            start, end = re.findall(r'\d+', row.find('small').text)
            if (int(start) >= 2005) and (int(end) <= 2019):
                out.append([row.find('a').text, end, row.find('a').get('href')])

    return out

In [12]:
# Obtain all players who played NCAA between 2005-2019
allURL = []
for char in string.ascii_lowercase:
    allURL.extend(getPlayersHTML(char))

In [13]:
# If first name and last name found, then we can record the players
found = []
for i, row in enumerate(notFound):
    name = row[1]
    split_name = name.replace('.', '').split(' ')
    first, last = split_name[0], split_name[1]
    for j in allURL:
        counter = 0
        if (first in j[0]) and (last in j[0]) and str(row[0]) == j[1]:
            temp = row.copy()
            temp.append(j[2])
            found.append(temp)
            counter += 1
        if counter >= 2:
            print(j)

In [14]:
len(found)

48

In [15]:
found[0]

[2009, 'Byron Mullens', 20, 'Top30', 24, '/cbb/players/bj-mullens-1.html']

In [16]:
out = []
for player in found:
    name, age, labelCat, labelPick, url = player[1], player[2], player[3], player[4], 'https://www.sports-reference.com' + player[5]
    try:
        row = scraping.scrapeRow(url)
        out.append([row[0], name, age] + row[2:] + [labelCat, labelPick])
    except:
        print(player)

[2009, 'Byron Mullens', 20, 'Top30', 24, '/cbb/players/bj-mullens-1.html']


In [17]:
data.extend(out)
len(data)

1525

**Retrieved the maximum number players possible, 1525**

**Save the output to a df now.**

In [21]:
colNames = scraping.getColNames('https://www.sports-reference.com/cbb/players/zion-williamson-1.html')
df = pd.DataFrame(data, columns= colNames)
df.to_csv('data/out.csv', index=False)

In [22]:
df.head()

,ID,Name,Age,Position,Height,Weight,Season,School,Conf,G,...,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/40,DraftCategory,DraftPick
0,andrew-bogut-1,Andrew Bogut,20,Forward,213,111,2005,Utah,MWC,35,...,,4.2,14.8,29.7,6.4,4.5,10.9,.358,Top15,1
1,marvin-williams-1,Marvin Williams,19,Forward,206,104,2005,UNC,ACC,36,...,,1.4,14.7,21.1,2.6,2.1,4.8,.239,Top15,2
2,deron-williams-1,Deron Williams,21,Guard,190,95,2005,Illinois,Big Ten,39,...,,0.5,19.3,22.5,2.1,2.5,4.6,.140,Top15,3
3,chris-paul-1,Chris Paul,20,Guard,183,79,2005,Wake Forest,ACC,32,...,,0.1,18.0,21.5,2.9,1.6,4.6,.171,Top15,4
4,raymond-felton-1,Raymond Felton,21,Guard,185,89,2005,UNC,ACC,36,...,,0.6,24.7,20.4,2.7,2.5,5.1,.179,Top15,5


**Some of the positions are subsets of existing positions, so we edit the feature**

In [23]:
df.loc[df['Position'] == 'Guard-Forward', 'Position'] = 'Guard'
df.loc[df['Position'] == 'Forward-Guard', 'Position'] = 'Forward'
df.loc[df['Position'] == 'Forward-Center', 'Position'] = 'Forward'
df.loc[df['Position'] == 'Center-Forward', 'Position'] = 'Center'

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1525 entries, 0 to 1524
Data columns (total 54 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ID             1525 non-null   object
 1   Name           1525 non-null   object
 2   Age            1525 non-null   int64 
 3   Position       1525 non-null   object
 4   Height         1525 non-null   object
 5   Weight         1525 non-null   object
 6   Season         1525 non-null   object
 7   School         1525 non-null   object
 8   Conf           1525 non-null   object
 9   G              1525 non-null   object
 10  GS             1525 non-null   object
 11  MP             1525 non-null   object
 12  FG             1525 non-null   object
 13  FGA            1525 non-null   object
 14  FG%            1525 non-null   object
 15  2P             1525 non-null   object
 16  2PA            1525 non-null   object
 17  2P%            1525 non-null   object
 18  3P             1525 non-null

**Change the DraftCategory to integers for Torch**

In [26]:
from sklearn.preprocessing import LabelEncoder

lb_make = LabelEncoder()
target = lb_make.fit_transform(df['DraftCategory'])
df['DraftCategory'] = target

**One-Hot Encoding for Position and Conf**

In [29]:
positions = pd.get_dummies(df['Position'])
conf = pd.get_dummies(df['Conf'])
df = pd.concat([df[['ID', 'Name', 'Age']], 
                positions, conf, 
                df[['Height', 'Weight', 'Season']],
                df.iloc[:, 9:54]],
                axis = 1)

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1525 entries, 0 to 1524
Data columns (total 89 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ID             1525 non-null   object
 1   Name           1525 non-null   object
 2   Age            1525 non-null   int64 
 3   Center         1525 non-null   uint8 
 4   Forward        1525 non-null   uint8 
 5   Guard          1525 non-null   uint8 
 6   A-10           1525 non-null   uint8 
 7   A-Sun          1525 non-null   uint8 
 8   AAC            1525 non-null   uint8 
 9   ACC            1525 non-null   uint8 
 10  AEC            1525 non-null   uint8 
 11  Big 12         1525 non-null   uint8 
 12  Big East       1525 non-null   uint8 
 13  Big Sky        1525 non-null   uint8 
 14  Big South      1525 non-null   uint8 
 15  Big Ten        1525 non-null   uint8 
 16  Big West       1525 non-null   uint8 
 17  CAA            1525 non-null   uint8 
 18  CUSA           1525 non-null

**Change the type of the statistics from string to numeric**

In [32]:
for col in df.columns[41: ]:
    df[col] = pd.to_numeric(df[col], errors='coerce')

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1525 entries, 0 to 1524
Data columns (total 89 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ID             1525 non-null   object 
 1   Name           1525 non-null   object 
 2   Age            1525 non-null   int64  
 3   Center         1525 non-null   uint8  
 4   Forward        1525 non-null   uint8  
 5   Guard          1525 non-null   uint8  
 6   A-10           1525 non-null   uint8  
 7   A-Sun          1525 non-null   uint8  
 8   AAC            1525 non-null   uint8  
 9   ACC            1525 non-null   uint8  
 10  AEC            1525 non-null   uint8  
 11  Big 12         1525 non-null   uint8  
 12  Big East       1525 non-null   uint8  
 13  Big Sky        1525 non-null   uint8  
 14  Big South      1525 non-null   uint8  
 15  Big Ten        1525 non-null   uint8  
 16  Big West       1525 non-null   uint8  
 17  CAA            1525 non-null   uint8  
 18  CUSA    

**5 recorsd have no OWS and DWS. Remove them**

In [34]:
df = df[df['OWS'].isna() == False]

**We already have FG, FGA. Thus can remove FG%. Same for 2P, 3P and FT**

In [35]:
df = df.drop(columns = ['FG%', '2P%', '3P%', 'FT%'])

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1520 entries, 0 to 1524
Data columns (total 85 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ID             1520 non-null   object 
 1   Name           1520 non-null   object 
 2   Age            1520 non-null   int64  
 3   Center         1520 non-null   uint8  
 4   Forward        1520 non-null   uint8  
 5   Guard          1520 non-null   uint8  
 6   A-10           1520 non-null   uint8  
 7   A-Sun          1520 non-null   uint8  
 8   AAC            1520 non-null   uint8  
 9   ACC            1520 non-null   uint8  
 10  AEC            1520 non-null   uint8  
 11  Big 12         1520 non-null   uint8  
 12  Big East       1520 non-null   uint8  
 13  Big Sky        1520 non-null   uint8  
 14  Big South      1520 non-null   uint8  
 15  Big Ten        1520 non-null   uint8  
 16  Big West       1520 non-null   uint8  
 17  CAA            1520 non-null   uint8  
 18  CUSA    

In [37]:
df.to_csv('data/full.csv', index=False)

**Different subsets of the original data, as some columns have missing values.**

In [38]:
mostSpecific = df[df.notnull().all(1)]
mostSpecific = mostSpecific.loc[:, (mostSpecific != mostSpecific.iloc[0]).any()]

In [39]:
secondMost = df.drop(columns = ['PER', 'PProd', 'ORB%', 'DRB%', 'STL%'])
secondMost = secondMost[secondMost.notnull().all(1)]
secondMost = secondMost.loc[:, (secondMost != secondMost.iloc[0]).any()]

In [40]:
leastSpecific = df.drop(columns = ['PER', 'PProd', 'ORB%', 'DRB%', 'STL%', 'TRB%', 'AST%', 'BLK%', 'USG%'])
leastSpecific = leastSpecific.loc[:, (leastSpecific != leastSpecific.iloc[0]).any()]

**Save these subsets for the model**

In [41]:
mostSpecific.to_csv('data/mostSpecific.csv', index=False)
secondMost.to_csv('data/secondMost.csv', index=False)
leastSpecific.to_csv('data/leastSpecific.csv', index=False)